In [1]:
#based on https://stackoverflow.com/questions/43101497/how-do-stream-data-to-a-bokeh-plot-in-jupyter-with-a-high-refresh-rate

import time
import os
import signal
from threading import Thread, Event

from sh import tail, cat

import numpy as np
import pmt

from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook, show, push_notebook

output_notebook()

Loading BokehJS ...

In [7]:
height = 500
width = 800

fig_power = figure(
        height=height,
        width=width,
        title="PowerSensor3 measurements",
        y_range=(0, 10),
    )
fig_power.xaxis.axis_label = "Time (s)"
fig_power.yaxis.axis_label = "Power (W)"

fig_freq = figure(
        height=height,
        width=width,
        title="GPU frequency",
        y_range=(0, 1000),
    )
fig_freq.xaxis.axis_label = "Time (s)"
fig_freq.yaxis.axis_label = "GPU Frequency (MHz)"

data = ColumnDataSource(data={'time': [], 'power_ps': [], 'power_tegra_cpu': [], 'power_tegra_gpu': [], 'gpu_freq': []})

line_ps = fig_power.line("time", "power_ps", source=data)
line_tegra_cpu = fig_power.line("time", "power_tegra_cpu", source=data, color='red')
line_tegra_gpu = fig_power.line("time", "power_tegra_gpu", source=data, color='green')
line_gpu = fig_freq.line("time", "gpu_freq", source=data)

handle = show(row(fig_power, fig_freq), notebook_handle=True)

In [3]:
event = Event()

def blocking_callback(_, event):
    n_sec_to_show = 30
    # powersensor dumpefile freq reduced by a factor 2000: so 10 Hz
    ps_freq = 10
    n_show = n_sec_to_show * ps_freq

    tegra = pmt.get_pmt('tegra')
    tegra.read()

    # wait for file to exist
    fname = '/dev/shm/ps3.txt'
    timeout = 9999  # seconds
    tstart = time.time()
    while not os.path.exists(fname):
        event.wait(.1)
        if time.time() - tstart > timeout:
            print(f"File {fname} still does not exist, giving up")
            return

    print("Starting read")

    process = tail('-f', '/dev/shm/ps3.txt', _iter=True)
    for line in process:
        if not line.startswith('S'):
            continue
        line = line.split(' ')
        t = float(line[1])
        p = float(line[-1])

        gpu_freq = int(cat('/sys/class/devfreq/57000000.gpu/cur_freq')) // 1000000

        state = tegra.read()
        power_tegra_gpu = state.watts(2)
        power_tegra_cpu = state.watts(3)
        
        data.stream({'time': [t], 'power_ps': [p], 'power_tegra_gpu': [power_tegra_gpu], 'power_tegra_cpu': [power_tegra_cpu], 'gpu_freq': [gpu_freq]}, n_show)

        push_notebook(handle=handle)
        if event.is_set():
            process.signal(signal.SIGINT)
            break
            
    print("exit")

thread = Thread(target=blocking_callback, args=(id, event))
thread.start()
# blocking_callback(id, event)

correct Create
argument is nullptr
tegra tegra
Starting read


In [4]:
# event.set()